# Análise exploratória Airbnb (Rio de Janeiro)
**Autor:** Douglas Trajano

Este notebook irá reunir a análise exploratória feita nos dados do Airbnb (Rio de Janeiro).

O objetivo deste projeto é desenvolver um classificador para predizer o `room_type` de cada anúncio no Airbnb para o Rio de Janeiro. Por isso, muitas visualizações serão focadas em entender o comportamento dos dados perante essa variável.

A estrutura completa do projeto pode ser vista [aqui](https://github.com/DougTrajano/ds_airbnb_rio).

**Disclaimer** ¯\\_(ツ)_/¯

- Não há uma explicação sobre cada *coluna/feature* na origem dos dados, então eventualmente pode não ser possível explicar o conceito de cada *coluna/feature*.
- Os gráficos desse notebook podem não aparecer no www.github.com, por isso, as imagens dos gráficos também foram fornecidas na pasta `/images` deste repositório.

## / imports

In [15]:
import pandas as pd
import numpy as np
import pandas_profiling as pdp
import processing_script

# graphs
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

## / load dataset

In [16]:
df_listings = processing_script.get_data(origin="listings")
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33715 entries, 0 to 33714
Columns: 106 entries, id to reviews_per_month
dtypes: float64(24), int64(21), object(61)
memory usage: 27.3+ MB


## / Pandas Profiling

Eu gosto de gerar o ProfileReport através da biblioteca Pandas Profiling, pois é possível ter uma visão inicial do dataset muito mais rápido e útil para evoluirmos na nossa análise exploratória.

In [3]:
from IPython.core.display import HTML

profile = pdp.ProfileReport(df_listings, title='Airbnb RJ - listings.csv')
profile.to_file(output_file="airbnb_data_report.html")

HTML('<a href="airbnb_data_report.html" target="_blank">airbnb_data_report.html</a>')

## / Análise exploratória

Após analisar o arquivo gerado pelo Pandas Profiling, identificamos que existem muitas colunas com apenas um valor, com todos os registros nulos ou grande parte dos registros nulos.

Isso provavelmente não agregará ao nosso modelo, mais pra frente iremos remover as colunas que não forem relevantes.

Agora vamos entender melhor os dados visualizando alguns gráficos.

### / Distribuição de room_types

In [3]:
df_temp = df_listings["room_type"].value_counts()

fig = {
  "data": [
    {
      "values": df_temp.values,
      "labels": df_temp.index,
      "domain": {"x": [0, .48]},
      "hole": .7,
      "type": "pie"
    },
    
    ],
  "layout": {
        "title":"Distribuição de room_types",
    }
}

fig = go.Figure(fig)
fig.update_layout(template="seaborn")
iplot(fig)

### / Distribuição de accommodates

In [4]:
room_types = df_listings["room_type"].unique().tolist()

df_accommodates = df_listings[["room_type", "accommodates"]].to_dict()
df_accommodates = pd.DataFrame(df_accommodates)

for i in range(len(df_accommodates)):
    if df_accommodates["accommodates"].loc[i] > 10:
        df_accommodates["accommodates"].loc[i] = "more than 10"

x_values = df_accommodates.groupby("accommodates").count().index.tolist()
x_values

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 'more than 10']

In [5]:
df_temp = df_accommodates.merge(pd.get_dummies(df_accommodates["room_type"]), left_index=True, right_index=True)
df_temp = df_temp.groupby("accommodates").sum()
df_temp

,Entire home/apt,Hotel room,Private room,Shared room
accommodates,,,,
1,113.0,39.0,1505.0,200.0
2,2859.0,110.0,5349.0,173.0
3,2379.0,34.0,888.0,62.0
4,7977.0,80.0,613.0,85.0
5,2649.0,18.0,114.0,37.0
6,4363.0,18.0,135.0,65.0
7,775.0,0.0,16.0,11.0
8,1305.0,4.0,43.0,21.0
9,194.0,4.0,10.0,5.0


In [6]:
data_vis = []

for room_type in room_types:
    y_values = df_temp[room_type].tolist()
    temp = go.Bar(name=room_type, x=x_values, y=y_values)
    data_vis.append(temp)
    
fig = go.Figure(data=data_vis)
fig.update_layout(barmode='group', xaxis=dict(tickmode = 'array',
                                              tickvals = list(range(1, len(x_values)+1)),
                                              ticktext = [str(each) for each in x_values]),
                  title="Distribuição de accommodates / room_type", template="seaborn")
fig.show()

### / Seleção das features mais prováveis para o modelo

Utiliando o <a href="airbnb_data_report.html" target="_blank">airbnb_data_report.html</a> vamos avaliar qualitativamente quais poderão ser as features mais importantes para o modelo, com base nelas, iremos construir mais alguns gráficos.

Features irrelevantes que poderão ser removidas:

```
city, calendar_updated, bed_type, availability_60, availability_90, availability_365, calendar_last_scraped, calculated_host_listings_count_entire_homes, country, country_code, experiences_offered, first_review, has_availability, host_acceptance_rate, host_has_profile_pic, host_id, host_location, host_name, host_picture_url, host_since, host_thumbnail_url, host_total_listings_count, host_url, id, interaction, is_business_travel_ready, jurisdiction_names, last_review, last_scraped, latitude, longitude, license, listing_url, market, maximum_minimum_nights, maximum_nights, maximum_nights_avg_ntm, medium_url, minimum_maximum_nights, minimum_minimum_nights, minimum_nights, minimum_nights_avg_ntm, neighborhood_overview, neighbourhood_cleansed, neighbourhood_group_cleansed, notes, number_of_reviews, number_of_reviews_ltm, picture_url, require_guest_phone_verification, require_guest_profile_picture, requires_license, review_scores_accuracy, review_scores_checkin, review_scores_cleanliness, review_scores_communication, review_scores_location, review_scores_rating, review_scores_value, reviews_per_month, scrape_id, smart_location, space, square_feet, state, street, summary, thumbnail_url, transit, xl_picture_url, zipcode
```


Features relevantes:

- bedrooms
- beds
- bathrooms
- cleaning_fee
- cancellation_policy
- calculated_host_listings_count
- calculated_host_listings_count_private_rooms
- calculated_host_listings_count_shared_rooms
- extra_people
> Precisa ser transformada para gerar insights
- amenities (dict)
> Precisa ser transformada para gerar insights
- availability_30
> Precisa ser transformada para gerar insights
- access (str)
> Precisa ser transformada para gerar insights
- description
> Precisa ser transformada para gerar insights
- guests_included
- host_about
> Precisa ser transformada para gerar insights
- host_identity_verified
- host_is_superhost
- host_listings_count
- host_neighbourhood
- host_response_rate
> Precisa ser transformada para gerar insights
- host_response_time
- host_verifications
> Precisa ser transformada para gerar insights
- house_rules
> Precisa ser transformada para gerar insights
- instant_bookable
- is_location_exact
- maximum_maximum_nights
- monthly_price
> Precisa ser transformada para gerar insights
- name
> Precisa ser transformada para gerar insights
- neighbourhood
- price
- property_type
- security_deposit
> Precisa ser transformada para gerar insights
- weekly_price

### / Distribuição de property_type

In [7]:
df_temp = df_listings[["property_type", "room_type", "host_is_superhost"]].groupby(["room_type", "property_type"]).count()
df_temp = pd.DataFrame(df_temp).reset_index()

y_values = df_temp["property_type"].unique().tolist()

temp = {}
for each in y_values:
    y_lst = []
    for room_type in room_types:
        try:
            value = df_temp[(df_temp["property_type"] == each) & 
                            (df_temp["room_type"] == room_type)]["host_is_superhost"].values[0]
        except:
            value = 0
        finally:
            y_lst.append(value)
    temp[each] = y_lst

fig = go.Figure()

for each in y_values:
    fig.add_trace(go.Bar(x=room_types, y=temp[each], name=each))

fig.update_layout(barmode='stack', template="seaborn", title="Distribuição de property_type / room_type")
fig.show()

In [8]:
df = processing_script.processing(df_listings, del_features=False, cat_features=False, fillna=False)
print(df.shape)
df.info()

100%|██████████████████████████████████████████████████████████████████████████| 33715/33715 [00:05<00:00, 6226.84it/s]


(33715, 218)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33715 entries, 0 to 33714
Columns: 218 entries, name to amenities_hbo_go
dtypes: float64(196), int64(11), object(11)
memory usage: 56.1+ MB


## / Features categoricas

Abaixo iremos identificar as features categóricas com menos de 10 valores únicos, essas receberão o tratamento do LabelEncoder.

As features que possuirem mais de 10 valores únicos serão deletadas no momento, pois em sua maioria, são campos de texto onde existe a presença do `room_type` escrito.

Como resultado dessa análise, iremos criar uma função para mapear essas features e depois, processá-las na função `processing`.

In [9]:
cat_features = []
del_features = []

print("Features que serão deletadas:")
for col in df.columns:
    if df[col].dtype == np.dtype("O"):
        n_values = df[col].nunique()
        if n_values > 10:
            print('A coluna: "{}" possui {} valores únicos.'.format(col, n_values))
            del_features.append(col)
        else:
            cat_features.append(col)
print("\n")
print("Features que serão tratadas através do LabelEncoder:")
print(cat_features)

Features que serão deletadas:
A coluna: "name" possui 32590 valores únicos.
A coluna: "description" possui 32052 valores únicos.
A coluna: "access" possui 14177 valores únicos.
A coluna: "house_rules" possui 15597 valores únicos.
A coluna: "host_about" possui 9571 valores únicos.
A coluna: "host_neighbourhood" possui 164 valores únicos.
A coluna: "neighbourhood" possui 99 valores únicos.


Features que serão tratadas através do LabelEncoder:
['host_response_time', 'property_type', 'room_type', 'cancellation_policy']


## / LabelEncoder para as features categóricas

Optei por criar minha própria função de LabelEncoder ao invés de usar a função nativa do sklearn. A justificativa é que consigo ter o mapa de transformação das features, o que seria necessário caso queira colocar essa predição em um endpoint para predições individuais, ao invés de todo o dataset.

In [10]:
processing_script.create_encoder(df, cat_features)

{'host_response_time': {'within an hour': 0,
  'within a day': 1,
  'within a few hours': 2,
  'a few days or more': 3,
  'nan': 4},
 'property_type': {'Condominium': 0,
  'Apartment': 1,
  'Loft': 2,
  'House': 3,
  'Others': 4,
  'Guest suite': 5},
 'room_type': {'Entire home/apt': 0,
  'Private room': 1,
  'Shared room': 2,
  'Hotel room': 3},
 'cancellation_policy': {'strict_14_with_grace_period': 0,
  'moderate': 1,
  'flexible': 2,
  'super_strict_30': 3,
  'super_strict_60': 4,
  'strict': 5}}

In [11]:
df_listings = processing_script.processing(df_listings, del_features=True, cat_features=True, fillna=True)
print(df_listings.shape)
df_listings.info()

100%|██████████████████████████████████████████████████████████████████████████| 33715/33715 [00:22<00:00, 1499.84it/s]


(33715, 211)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33715 entries, 0 to 33714
Columns: 211 entries, host_response_time to amenities_hbo_go
dtypes: float64(197), int64(14)
memory usage: 54.3 MB


## / Principais features correlacionadas com room_type

In [12]:
#Correlation with TARGET variable
cor = df_listings.corr()
cor_target = abs(cor["room_type"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.05].sort_values(ascending=False)
relevant_features = pd.DataFrame(relevant_features[1:])
relevant_features.head(10)

,room_type
calculated_host_listings_count_private_rooms,0.376877
accommodates,0.360498
bedrooms,0.305256
calculated_host_listings_count_shared_rooms,0.293435
amenities_air_conditioning,0.283112
property_type,0.272772
amenities_lock_on_bedroom_door,0.262164
amenities_breakfast,0.255286
amenities_elevator,0.222050
amenities_kitchen,0.221026


In [13]:
df_temp = relevant_features["room_type"]

fig = {
  "data": [
    {
      "y": df_temp.values,
      "x": df_temp.index,
      "type": "bar"
    },
    
    ],
  "layout": {
        "title":"Principais features correlacionadas com room_type",
    }
}

fig = go.Figure(fig)
fig.update_layout(template="seaborn")
iplot(fig)

In [14]:
file_path = "./data/listings.csv"
df_listings.to_csv(file_path, index=False, encoding="utf-8")
print("Arquivo salvo:", file_path)

Arquivo salvo: ./data/listings.csv


## / Conclusões

Neste notebook nós tivemos uma visão geral dos dados e também criamos as funções que processam os dataset `listings.csv` que será utilizado para treinar o modelo que irá prever o `room_type` dos futuros anúncios.

Nos vemos no próximo notebook (modeling.ipynb). =)